In [31]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import openpyxl
import os
import time
import math
import webbrowser
import numpy as np
import re
import sys
from pandas import DataFrame


In [32]:
chrome_path = "C:\Temp\chromedriver.exe"
options = webdriver.ChromeOptions()
#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
# 창 숨기는 옵션 추가
# options.add_argument("headless")

driver = webdriver.Chrome(chrome_path,options=options)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", 
{"source": """ Object.defineProperty(navigator, 'webdriver', { get: () => undefined }) """})
url = "https://www.oliveyoung.co.kr/store/main/main.do?oy=0"
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()

C:\Users\won\AppData\Local\Temp\ipykernel_10788\94386705.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path,options=options)


In [33]:
prod_text = '선케어'
driver.find_element(By.CLASS_NAME,"placeholder_area").click()
prod = driver.find_element(By.ID,"query")
prod.send_keys(prod_text)
time.sleep(1)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "searchSubmit"))).click()

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [34]:
#선케어 제품들  url 가져오기
suncare_url_list = []
items = soup.find_all("div",attrs={"class":"prd_info"})
for item in items:
    url=item.find('a')['href']
    suncare_url_list.append(url)


In [ ]:
for i in range(0,10):  
    driver.get(suncare_url_list[i])
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "reviewInfo"))).click()
    time.sleep(2)


    html_type = driver.page_source
    soup_type = BeautifulSoup(html_type,'html.parser')
    time.sleep(2)
    #상품이름 가져오기
    pos_prod_name_list=[]
    pos_prod_name=soup_type.find('p',attrs={'class':'prd_name'}).get_text()
    
    #상품 이미지 저장하기
    try :
        prod_url=soup_type.find('img',attrs={'id':'mainImg'})['src']
        print(prod_url)
    except :
        pass
    else :
        img_name = 'C:/Users/won/teamproject/prod_img/'+'sunitem'+ str(i)+'.jpg'
        urllib.request.urlretrieve(prod_url,img_name)  
        time.sleep(3)


    #페이지 넘기기면서 데이터 가져오기(리뷰,평점,아이디)
    pos_num=1
    pos_user_name_list=[]
    pos_rate_list=[]
    pos_review_list=[]
    # postive_reviews=['//*[@id="searchPoint"]/li[2]','//*[@id="searchPoint"]/li[3]']
    # negative_reviws=['//*[@id="searchPoint"]/li[2]','//*[@id="searchPoint"]/li[2]']
    #체험단 제거
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID,'searchType_1'))).click()
    time.sleep(1)
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID,'searchType_3'))).click()
    time.sleep(1)
    #리뷰 검색 필터
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,'btnOption'))).click()
    #5점
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[2]'))).click()
    time.sleep(2)
    #4점

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[3]'))).click()    
    time.sleep(2)
    #확인 버튼
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))).click()
    time.sleep(2)

    html_type = driver.page_source
    soup_type = BeautifulSoup(html_type,'html.parser')
    time.sleep(2)

    for num in range(1,11):
        #사용자이름

        try:
            user_names=soup_type.find_all('a',attrs={'class':'id'})
            for user_name in user_names:
                pos_user_name_list.append(user_name.get_text())
        except:        
            pos_user_name_list.append("err")

        #평점
        try:
            non_rates=soup_type.find_all('div',attrs={'class':'score_area'})
            for rates in non_rates:
                rate = rates.find('span',attrs={'class':'point'}).get_text()
                rate = re.findall(r'\d', rate)
                pos_rate_list.append(rate[1])

        except:
            pos_rate_list.append("rate_err")

        #리뷰
        try:    
            reviews=soup_type.find_all('div',attrs={'class':'txt_inner'})
            for review in reviews:
                review = re.compile('[가-힣]+').findall(review.get_text())
                pos_review_list.append(review)
        except:
            pos_review_list.append("review_err")
        try:

            page_element=soup_type.find('div',attrs={'class':'pageing'})
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "{}".format(num+1)))).click()
            time.sleep(1)
            html_type = driver.page_source
            soup_type = BeautifulSoup(html_type,'html.parser')
            time.sleep(2)
        except:
            break

    #리뷰 필터 버튼
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,'btnOption'))).click()

    #초기화 버튼
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'resetBtn'))).click()

    #팝업창 확인 버튼
    from selenium.webdriver.common.alert import Alert
    da = Alert(driver)
    da.accept()


    #리뷰 검색 필터
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME,'btnOption'))).click()
    time.sleep(1)
    #2점
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[5]'))).click()
    time.sleep(1)
    #1점
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="searchPoint"]/li[6]'))).click()
    time.sleep(1)
    #확인 버튼
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'btnFilterConfirm'))).click()
    time.sleep(1)

    html_type = driver.page_source
    soup_type = BeautifulSoup(html_type,'html.parser')
    time.sleep(1)
    neg_num=1
    neg_user_name_list=[]
    neg_rate_list=[]
    neg_review_list=[]

    #상품이름 가져오기
    neg_prod_name_list=[]
    neg_prod_name=soup_type.find('p',attrs={'class':'prd_name'}).get_text()


    for num in range(1,11):
        #사용자이름

        try:
            user_names=soup_type.find_all('a',attrs={'class':'id'})
            for user_name in user_names:
                neg_user_name_list.append(user_name.get_text())
        except:        
            neg_user_name_list.append("err")

        #평점
        try:
            non_rates=soup_type.find_all('div',attrs={'class':'score_area'})
            for rates in non_rates:
                rate = rates.find('span',attrs={'class':'point'}).get_text()
                rate = re.findall(r'\d', rate)
                neg_rate_list.append(rate[1])

        except:
            neg_rate_list.append("rate_err")

        #리뷰
        try:    
            reviews=soup_type.find_all('div',attrs={'class':'txt_inner'})
            for review in reviews:
                review = re.compile('[가-힣]+').findall(review.get_text())
                neg_review_list.append(review)
        except:
            neg_review_list.append("review_err")
            
            
        try:
            
            page_element=soup_type.find('div',attrs={'class':'pageing'})

            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "{}".format(num+1)))).click()
            time.sleep(1)
            
        except:
            break

    ### 리뷰 가져오기
    try:
        pos_data={"상품이름" : pos_prod_name,
                 "유저 ID" : pos_user_name_list,
                  "유저평점" : pos_rate_list,
                  '리뷰':pos_review_list

                 }

        pos_df = pd.DataFrame(pos_data)
        pos_fileName = "sunitem"+ str(i) + 'pos_sunscreen.csv'
        os.chdir('C:/Users/won/teamproject')
        pos_df.to_csv(pos_fileName, index=False, encoding='cp949')
    except:
        print(pos_prod_name)
        pass
        

    try:
        neg_data={"상품이름" : neg_prod_name,
                 "유저 ID" : neg_user_name_list,
                  "유저평점" : neg_rate_list,
                  '리뷰':neg_review_list

                 }

        neg_df = pd.DataFrame(neg_data)
        neg_fileName = "sunitem"+ str(i) + 'neg_sunscreen.csv'

        os.chdir('C:/Users/won/teamproject')
        neg_df.to_csv(neg_fileName, index=False, encoding='cp949')
        
    except:
        print(neg_prod_name)
        pass

https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0018/A00000018434702ko.jpg?l=ko
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0016/A00000016629514ko.jpg?l=ko
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0017/A00000017912722ko.jpg?l=ko
https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0017/A00000017122206ko.jpg?l=ko
